In [7]:
import json
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np 


In [8]:
data = []
with open('imdb_movies_2000to2022.prolific.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))
df = pd.DataFrame(data)

In [9]:
# print(df[:5])

     imdb_id  ...                        rating
0  tt0035423  ...  {'avg': 6.4, 'votes': 85923}
1  tt0088751  ...    {'avg': 5.3, 'votes': 328}
2  tt0096056  ...    {'avg': 5.6, 'votes': 830}
3  tt0113092  ...    {'avg': 3.4, 'votes': 829}
4  tt0116391  ...    {'avg': 6.2, 'votes': 257}

[5 rows x 7 columns]


In [10]:
df.replace({'\\N':np.nan},inplace=True)
def get_avg_Rating(x):
    if isinstance(x, dict):
        return x.get('avg', np.nan)
    return np.nan
df['rating.avg'] = df['rating'].apply(get_avg_Rating)
# print(df[:5])

     imdb_id                 title  ...                        rating rating.avg
0  tt0035423        Kate & Leopold  ...  {'avg': 6.4, 'votes': 85923}        6.4
1  tt0088751     The Naked Monster  ...    {'avg': 5.3, 'votes': 328}        5.3
2  tt0096056  Crime and Punishment  ...    {'avg': 5.6, 'votes': 830}        5.6
3  tt0113092         For the Cause  ...    {'avg': 3.4, 'votes': 829}        3.4
4  tt0116391                  Gang  ...    {'avg': 6.2, 'votes': 257}        6.2

[5 rows x 8 columns]


In [13]:
df['rating.avg'].fillna(df['rating.avg'].mean(),inplace=True)

/var/folders/bc/_h0z1r_n7xsczt7nbblx6sxr0000gn/T/ipykernel_16688/1796543545.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rating.avg'].fillna(df['rating.avg'].mean(),inplace=True)


In [15]:
df['rating.avg'].fillna(df['rating.avg'].mean(), inplace=True)
df['runtime'] = pd.to_numeric(df['runtime'])
df['runtime'].fillna(df['runtime'].median(), inplace=True)
df['year'] = pd.to_numeric(df['year'])
df['year'].fillna(df['year'].median(), inplace=True)

/var/folders/bc/_h0z1r_n7xsczt7nbblx6sxr0000gn/T/ipykernel_16688/160809515.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rating.avg'].fillna(df['rating.avg'].mean(), inplace=True)
/var/folders/bc/_h0z1r_n7xsczt7nbblx6sxr0000gn/T/ipykernel_16688/160809515.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [18]:
mlb = MultiLabelBinarizer()
genre_labels = mlb.fit_transform(df['genres'])
genre_df = pd.DataFrame(genre_labels, columns=mlb.classes_)
# print(genre_labels[:5])
# print(mlb.classes_)
# print(genre_df[:5])

[[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
['' 'Action' 'Adventure' 'Animation' 'Biography' 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Family' 'Fantasy' 'History' 'Horror' 'Music'
 'Musical' 'Mystery' 'News' 'Reality-TV' 'Romance' 'Sci-Fi' 'Short'
 'Sport' 'Thriller' 'War' 'Western']
      Action  Adventure  Animation  ...  Sport  Thriller  War  Western
0  0       0          0          0  ...      0         0    0        0
1  0       0          0          0  ...      0         0    0        0
2  0       0          0          0  ...      0         0    0        0
3  0       1          1          0  ...      0         0    0        0
4  0       1          0          0  ...      0         0    0        0

[5 rows x 25 columns]


In [19]:
df = pd.concat([df,genre_df], axis=1)
# print(df[:5])

     imdb_id                 title  year  ...  Thriller War Western
0  tt0035423        Kate & Leopold  2001  ...         0   0       0
1  tt0088751     The Naked Monster  2005  ...         0   0       0
2  tt0096056  Crime and Punishment  2002  ...         0   0       0
3  tt0113092         For the Cause  2000  ...         0   0       0
4  tt0116391                  Gang  2000  ...         0   0       0

[5 rows x 33 columns]


In [21]:
X = df[['runtime', 'year', 'rating.avg']]
y = genre_df

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [24]:
for depth in range(1,11):
    clf = DecisionTreeClassifier(max_depth=depth)

    clf.fit(X_train, y_train)
    test_pred = clf.predict(X_test)
    test_correct = []
    for i in range(len(test_pred)):
        test = test_pred[i]
        actual = y_test.values[i]
        # print(test)
        # print(actual)
        if any(test[j]==1 and actual[j]==1 for j in range(len(test))):
            test_correct.append(True)
        else:
            test_correct.append(False)
    test_accuracy = np.mean(test_correct)
    print(f'max_depth={depth} | Test_Accuracy: {test_accuracy:.2f}')

max_depth=1 | Test_Accuracy: 0.42
max_depth=2 | Test_Accuracy: 0.25
max_depth=3 | Test_Accuracy: 0.27
max_depth=4 | Test_Accuracy: 0.40
max_depth=5 | Test_Accuracy: 0.39
max_depth=6 | Test_Accuracy: 0.33
max_depth=7 | Test_Accuracy: 0.35
max_depth=8 | Test_Accuracy: 0.36
max_depth=9 | Test_Accuracy: 0.37
max_depth=10 | Test_Accuracy: 0.37


In [27]:
for depth in range(1,11):
    clf = DecisionTreeClassifier(max_depth=depth)

    clf.fit(X_train, y_train)
    train_pred = clf.predict(X_train)
    train_correct = []
    for i in range(len(train_pred)):
        train = train_pred[i]
        actual = y_train.values[i]
        # print(test)
        # print(actual)
        if any(train[j]==1 and actual[j]==1 for j in range(len(train))):
            train_correct.append(True)
        else:
            train_correct.append(False)
    train_accuracy = np.mean(train_correct)
    print(f'max_depth={depth} | Train_Accuracy: {train_accuracy:.2f}')

max_depth=1 | Train_Accuracy: 0.43
max_depth=2 | Train_Accuracy: 0.25
max_depth=3 | Train_Accuracy: 0.28
max_depth=4 | Train_Accuracy: 0.41
max_depth=5 | Train_Accuracy: 0.41
max_depth=6 | Train_Accuracy: 0.34
max_depth=7 | Train_Accuracy: 0.36
max_depth=8 | Train_Accuracy: 0.38
max_depth=9 | Train_Accuracy: 0.41
max_depth=10 | Train_Accuracy: 0.43


As max_depth increases, the accuracy decreases, as the model tries to memorize training data. 